In [1]:
import xarray as xr
import h5py
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
import cartopy
from datetime import datetime, timedelta
import os
import numpy as np
import pandas as pd
import imageio

In [2]:
grid_path = r'Z:/cluster_projects/ro/1149_10/earthformer-multisource-to-inca/data_adapted_condition_0.02_20_all/Auxillary/grid_2d.h5'

In [3]:
rivers = cartopy.feature.NaturalEarthFeature(category='physical', name='rivers_lake_centerlines', scale='10m')
border = cartopy.feature.NaturalEarthFeature('cultural', 'admin_0_countries', '50m')
lon_model = np.flipud(h5py.File(grid_path,'r')['lon'][:, :])
lat_model = np.flipud(h5py.File(grid_path,'r')['lat'][:, :])

In [9]:
# Load TAHMO station data (CSV format with columns:'precipitation')
tahmo_root_dir = r'C:\Users\c.kwa\Desktop\meteosat_retrieval\data_downloads\TAHMO\TAHMO_export_6704eaa37e81da18c0b7e245'


station_name_list_tahmo = []
station_data_list_TAHMO = []
for station in os.listdir(tahmo_root_dir):
    if station.endswith('.csv'):
        station_file = os.path.join(tahmo_root_dir, station)
        station_data = pd.read_csv(station_file)
        station_name_list_tahmo.append(station.split('.')[0])
        station_data_list_TAHMO.append(station_data)

station_data_tahmo_dict = dict(zip(station_name_list_tahmo, station_data_list_TAHMO))

stations_meta_data_TAHMO = station_data_tahmo_dict.pop('stations_metadata')
stations_sensor_data = station_data_tahmo_dict.pop('sensors_metadata')

display(stations_meta_data_TAHMO)
display(stations_sensor_data)

,station code,name,country,installation height (m),latitude,longitude,elevation (m),timezone
0,TA00005,Asankragwa SHS,GH,2.0,5.807731,-2.426395,125.1,Africa/Accra
1,TA00007,Nana Yaa Kesse SHS Duayaw Nkwanta,GH,2.0,7.188273,-2.097477,341.1,Africa/Accra
2,TA00010,Chiraa SHS,GH,2.0,7.389595,-2.185991,337.1,Africa/Accra
3,TA00016,"Accra Academy School, Accra",GH,2.0,5.573104,-0.244500,32.4,Africa/Accra
4,TA00045,"Asesewaa Senior High School, Asesewaa",GH,2.0,6.400626,-0.146577,372.3,Africa/Accra
5,TA00113,Nkwanta SHS,GH,2.0,8.271124,0.515265,213.7,Africa/Accra
6,TA00116,Amedzofe Technical Institute,GH,2.0,6.845815,0.440698,731.8,Africa/Accra
7,TA00117,Keta SHS,GH,2.0,5.895083,0.989567,10.0,Africa/Accra
8,TA00118,Tema Secondary School,GH,2.0,5.641413,-0.011870,18.4,Africa/Accra
9,TA00120,Nkroful Agric SHS,GH,2.0,4.971861,-2.322676,28.0,Africa/Accra


,sensor code,sensor name,installation date,deinstallation date,port number,installation height (m)
0,S001402,ATMOS 41,2020-06-25T00:00:00Z,2199-01-01T00:00:00Z,1,2.0
1,S001188,ATMOS 41,2019-07-02T00:00:00Z,2020-06-25T00:00:00Z,1,2.0
2,S001233,ATMOS 41,2019-09-08T00:00:00Z,2199-01-01T00:00:00Z,1,2.0
3,S001234,ATMOS 41,2019-09-08T00:00:00Z,2023-06-04T17:00:00Z,1,2.0
4,S001944,ATMOS 41,2023-06-04T17:05:00Z,2199-01-01T00:00:00Z,1,2.0
5,S001566,ATMOS 41,2021-05-18T00:00:00Z,2023-07-12T00:00:00Z,1,2.0
6,S001212,ATMOS 41,2019-07-11T00:00:00Z,2021-05-18T00:00:00Z,1,2.0
7,S001955,ATMOS 41,2023-07-12T00:00:00Z,2199-01-01T00:00:00Z,1,2.0
8,S000711,ATMOS 41,2018-05-23T00:00:00Z,2199-01-01T00:00:00Z,1,2.0
9,S000127,ATMOS 41,2017-01-29T00:00:00Z,2020-09-14T00:00:00Z,1,2.0


In [4]:
# Load TAHMO station data (CSV format with columns:'precipitation')
gmet_root_dir = r'C:\Users\c.kwa\Desktop\meteosat_retrieval\data_downloads\GMET\GMet_AWSdata\GMet_AWSdata'

station_data_list_GMET = []
station_name_list = []
for station in os.listdir(gmet_root_dir):
    if station.endswith('.csv'):
        station_file = os.path.join(gmet_root_dir, station)
        station_data = pd.read_csv(station_file)
        station_data_list_GMET.append(station_data)
        station_name_list.append(station.split('.')[0])
    #display(station_data)

stations_meta_data_GMET = station_data_list_GMET.pop(7)
station_name_list = stations_meta_data_GMET['STN'].tolist()

station_data_gmet_dict = dict(zip(station_name_list, station_data_list_GMET))

display(stations_meta_data_GMET)

display(station_data_gmet_dict['Abetifi'])

,STN,LON,LAT
0,Abetifi,-0.747020,6.680400
1,Ada_Foah,0.617981,5.831201
2,Akim_Oda,-0.978370,5.929300
3,Akosombo,0.056591,6.291360
4,Akuse,0.119290,6.094600
5,Axim,-2.237005,4.864180
6,Enchi,-2.823720,5.823700
7,Jirapa,-2.719410,10.534580
8,Kalbeo,-0.887150,10.752900
9,Kpandai,-0.017880,8.484500


,Date,Time,Precipitation (mm)
0,1/1/2020,12:00:00 AM,0
1,1/1/2020,12:15:00 AM,0
2,1/1/2020,12:30:00 AM,0
3,1/1/2020,12:45:00 AM,0
4,1/1/2020,1:00:00 AM,0
...,...,...,...
135264,12/31/2023,10:45:00 PM,0
135265,12/31/2023,11:00:00 PM,0
135266,12/31/2023,11:15:00 PM,0
135267,12/31/2023,11:30:00 PM,0


In [5]:

# Define intervals and colors
vmin, vmax = 0, 154
intervals = [vmin, 0.2, 0.3, 0.6, 0.9, 1.7, 2.7, 5, 8.6, 15, 27.3, 50, 89.9, vmax]
colors = ['#fffffe', '#0101ff', '#0153ff', '#00acfe', '#01feff',
          '#8cff8c', '#fffe01', '#ff8d01', '#fe4300', '#f60100',
          '#bc0030', '#ad01ac', '#ff00ff']

# Create custom colormap and normalization
custom_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors)
custom_norm = BoundaryNorm(intervals, custom_cmap.N)

In [6]:
def generate_gif_imerg(imerg_dir, date, time_range, dataset_path, version, output_gif, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds=None, lon_bounds=None):
    timestamps = generate_timestamps(date, time_range)
    images = []
    
    for timestamp in timestamps:
        print(timestamp)
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
        
        file_path = construct_imerg_filepath(imerg_dir, timestamp, version)
        if os.path.exists(file_path):
            plot_data(ax, file_path, dataset_path, custom_cmap, custom_norm, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds, lon_bounds)
            if 'NRT'in imerg_dir:
                plt.title(f"IMERG-Early {timestamp.strftime('%H:%M')} UTC", fontsize=30)

            else:
                plt.title(f"IMERG-Final {timestamp.strftime('%H:%M')} UTC", fontsize=30)
            
            temp_image = f"temp_{timestamp.strftime('%Y%m%d%H%M')}.png"
            plt.savefig(temp_image)
            images.append(imageio.imread(temp_image))
            os.remove(temp_image)
        
        plt.close(fig)
    
    imageio.mimsave(output_gif, images, duration=5)

def generate_gif_model(model_dir, date, time_range, variable_name, output_gif, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds=None, lon_bounds=None):
    timestamps = generate_timestamps(date, time_range)
    images = []
    
    for timestamp in timestamps:
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
        
        file_path = os.path.join(model_dir, f"Combined_{(timestamp - timedelta(seconds=1)).strftime('%Y%m%d%H%M%S')}.h.h5")
        if os.path.exists(file_path):
            plot_data_model(ax, file_path, variable_name, custom_cmap, custom_norm, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds, lon_bounds)
            if '129'in model_dir:
                plt.title(f"Earthformer MSE loss {timestamp.strftime('%H:%M')} UTC", fontsize=30)

            else:
                plt.title(f"Earthformer Balanced loss {timestamp.strftime('%H:%M')} UTC", fontsize=30)

            
            temp_image = f"temp_{timestamp.strftime('%Y%m%d%H%M')}.png"
            plt.savefig(temp_image)
            images.append(imageio.imread(temp_image))
            os.remove(temp_image)
        
        plt.close(fig)
    
    imageio.mimsave(output_gif, images, duration=5)

def generate_timestamps(date, time_range):
    start_time = datetime.strptime(date, '%Y%m%d') + timedelta(hours=time_range[0])
    end_time = datetime.strptime(date, '%Y%m%d') + timedelta(hours=time_range[1])
    
    timestamps = []
    current_time = start_time
    while current_time <= end_time:
        timestamps.append(current_time)
        timestamps.append(current_time + timedelta(minutes=30))
        current_time += timedelta(hours=1)
    
    return timestamps

def time_to_minutes(time_str):
        time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')
        minutes = time_obj.hour * 60 + time_obj.minute - 30
        return f"{minutes:04d}" if minutes > 0 else "0000"

def construct_imerg_filepath(imerg_dir, timestamp, version):
    if timestamp.strftime('%H') == 0:
        if timestamp.minute == 0:
            imerg_time = timestamp -timedelta(seconds = 1)
            imerg_hour = '23'
            start_minute = '3'
            end_minute = '5959'
        else:
            imerg_time = timestamp
            imerg_hour = '00'
            start_minute = '0'
            end_minute = '2959'
    else:
        imerg_time = timestamp - timedelta(hours=1) if timestamp.minute == 0 else timestamp
        imerg_hour = imerg_time.strftime('%H')
        minutes = f"{imerg_time.minute:02d}00"
        
        # return os.path.join(
        #     imerg_dir,
        #     imerg_time.strftime('%Y/%m/%d'),
        #     f"3B-HHR.MS.MRG.3IMERG.{imerg_time.strftime('%Y%m%d')}-S{imerg_hour}0000-E{imerg_hour}5959.{minutes}.V07B.HDF5"
        # )
    
        hour_str = timestamp.strftime('%H:%M')
            
        minutes = time_to_minutes(timestamp.strftime("%Y-%m-%d %H:%M:%S"))
    
        if timestamp.minute == 0:
            end_minute = "5959"
            start_minute = '3'
            imerg_time = timestamp - timedelta(hours=1)
            imerg_hour = imerg_time.strftime('%H')
        else:
            end_minute = "2959"
            start_minute = '0'
            imerg_time = timestamp #- timedelta(hours=1)
            imerg_hour = imerg_time.strftime('%H')

    if 'NRT' in imerg_dir:
        imerg_file = os.path.join(
            imerg_dir,
            timestamp.strftime('%Y/%m/%d'),
            f"3B-HHR-E.MS.MRG.3IMERG.{imerg_time.strftime('%Y%m%d')}-S{imerg_hour}{start_minute}000-E{imerg_hour}{end_minute}.{minutes}.V06{version}.HDF5"
        )
    else:
        imerg_file = os.path.join(
            imerg_dir,
            timestamp.strftime('%Y/%m/%d'),
            f"3B-HHR.MS.MRG.3IMERG.{imerg_time.strftime('%Y%m%d')}-S{imerg_hour}{start_minute}000-E{imerg_hour}{end_minute}.{minutes}.V07B.HDF5"
        )

    return imerg_file

    



    

def plot_data(ax, file_path, dataset_path, cmap, norm, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds, lon_bounds):
    with h5py.File(file_path, 'r') as f:
        data = f[dataset_path][:]
        lon = f['Grid/lon'][:]
        lat = f['Grid/lat'][:]
        data = np.flipud(np.squeeze(data.T))
        
        if lon.ndim == 1 and lat.ndim == 1:
            lon, lat = np.meshgrid(lon, lat)
            lat = np.flipud(lat)

        mesh = ax.pcolormesh(lon, lat, data, cmap=cmap, norm=norm, transform=ccrs.PlateCarree(), shading='auto')
        
        if lat_bounds and lon_bounds:
            ax.set_extent([lon_bounds[0], lon_bounds[1], lat_bounds[0], lat_bounds[1]], crs=ccrs.PlateCarree())
        
        ax.add_feature(cfeature.BORDERS, linestyle='-', edgecolor='black')
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        # Customize gridline label font size and padding
        gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {'size': 25, 'rotation': 0}  # Longitude label style
        gl.ylabel_style = {'size': 25, 'rotation': 0}
        #ax.set_title(title, fontsize = 25)

        # Add station markers
        if plot_tahmo:
            # Filter only the stations in the provided list
            selected_stations_tahmo = stations_meta_data_TAHMO[stations_meta_data_TAHMO["station code"].isin(tahmo_stations)]
            
            # Scatter plot for selected stations
            ax.scatter(selected_stations_tahmo["longitude"], selected_stations_tahmo["latitude"], 
                       color='black', s=120, label='TAHMO Stations', transform=ccrs.PlateCarree())
        
            # Add station labels
            for _, row in selected_stations_tahmo.iterrows():
                ax.text(row["longitude"] + 0.05, row["latitude"] + 0.05, row["station code"], 
                        fontsize=16, transform=ccrs.PlateCarree())

        if plot_gmet:
            # Filter only the stations in the provided list
            selected_stations_gmet = stations_meta_data_GMET[stations_meta_data_GMET["STN"].isin(gmet_stations)]
            
            # Scatter plot for selected stations
            ax.scatter(selected_stations_gmet["LON"], selected_stations_gmet["LAT"], 
                       color='black', s=120, label='GMET Stations', transform=ccrs.PlateCarree())
        
            # Add station labels
            for _, row in selected_stations_gmet.iterrows():
                ax.text(row["LON"] + 0.05, row["LAT"] + 0.05, row["STN"], 
                        fontsize=16, transform=ccrs.PlateCarree())

def plot_data_model(ax, file_path, variable_name, cmap, norm, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds, lon_bounds):
    ds = xr.open_dataset(file_path, engine='netcdf4')
    data = np.flipud(np.squeeze(ds[variable_name].values))
    lon = lon_model
    lat = lat_model
    mesh = ax.pcolormesh(lon, lat, data, cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
    
    
    if lat_bounds and lon_bounds:
        ax.set_extent([lon_bounds[0], lon_bounds[1], lat_bounds[0], lat_bounds[1]], crs=ccrs.PlateCarree())
    
    ax.add_feature(cfeature.BORDERS, linestyle='-', edgecolor='black')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    # Customize gridline label font size and padding
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 25, 'rotation': 0}  # Longitude label style
    gl.ylabel_style = {'size': 25, 'rotation': 0}
    #ax.set_title(title, fontsize = 25)

    if lat_bounds and lon_bounds:
        ax.set_extent([lon_bounds[0], lon_bounds[1], lat_bounds[0], lat_bounds[1]], crs=ccrs.PlateCarree())
    
    # Add station markers
    if plot_tahmo:
        # Filter only the stations in the provided list
        selected_stations_tahmo = stations_meta_data_TAHMO[stations_meta_data_TAHMO["station code"].isin(tahmo_stations)]
        
        # Scatter plot for selected stations
        ax.scatter(selected_stations_tahmo["longitude"], selected_stations_tahmo["latitude"], 
                   color='black', s=120, label='TAHMO Stations', transform=ccrs.PlateCarree())
    
        # Add station labels
        for _, row in selected_stations_tahmo.iterrows():
            ax.text(row["longitude"] + 0.05, row["latitude"] + 0.05, row["station code"], 
                    fontsize=16, transform=ccrs.PlateCarree())
    
    if plot_gmet:
        # Filter only the stations in the provided list
        selected_stations_gmet = stations_meta_data_GMET[stations_meta_data_GMET["STN"].isin(gmet_stations)]
        
        # Scatter plot for selected stations
        ax.scatter(selected_stations_gmet["LON"], selected_stations_gmet["LAT"], 
                   color='black', s=120, label='GMET Stations', transform=ccrs.PlateCarree())
    
        # Add station labels
        for _, row in selected_stations_gmet.iterrows():
            ax.text(row["LON"] + 0.05, row["LAT"] + 0.05, row["STN"], 
                    fontsize=16, transform=ccrs.PlateCarree())


In [7]:
def generate_gif_imerg2(imerg_dir, start_datetime, end_datetime, dataset_path, version, output_gif, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds=None, lon_bounds=None):
    timestamps = generate_timestamps2(start_datetime, end_datetime)
    images = []
    
    for timestamp in timestamps:
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
        
        file_path = construct_imerg_filepath(imerg_dir, timestamp, version)
        if os.path.exists(file_path):
            plot_data(ax, file_path, dataset_path, custom_cmap, custom_norm, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds, lon_bounds)
            if 'NRT' in imerg_dir:
                plt.title(f"IMERG-Early {timestamp.strftime('%H:%M')} UTC", fontsize=30)
            else:
                plt.title(f"IMERG-Final {timestamp.strftime('%H:%M')} UTC", fontsize=30)
            
            temp_image = f"temp_{timestamp.strftime('%Y%m%d%H%M')}.png"
            plt.savefig(temp_image)
            images.append(imageio.imread(temp_image))
            os.remove(temp_image)
        
        plt.close(fig)
    
    imageio.mimsave(output_gif, images, duration=5)


def generate_gif_model2(model_dir, start_datetime, end_datetime, variable_name, output_gif, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds=None, lon_bounds=None):
    timestamps = generate_timestamps2(start_datetime, end_datetime)
    images = []
    
    for timestamp in timestamps:
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
        
        file_path = os.path.join(model_dir, f"Combined_{(timestamp - timedelta(seconds=1)).strftime('%Y%m%d%H%M%S')}.h.h5")
        if os.path.exists(file_path):
            plot_data_model(ax, file_path, variable_name, custom_cmap, custom_norm, plot_tahmo, plot_gmet, tahmo_stations, gmet_stations, lat_bounds, lon_bounds)
            if '129' in model_dir:
                plt.title(f"Earthformer MSE loss {timestamp.strftime('%H:%M')} UTC", fontsize=30)
            else:
                plt.title(f"Earthformer Balanced loss {timestamp.strftime('%H:%M')} UTC", fontsize=30)
            
            temp_image = f"temp_{timestamp.strftime('%Y%m%d%H%M')}.png"
            plt.savefig(temp_image)
            images.append(imageio.imread(temp_image))
            os.remove(temp_image)
        
        plt.close(fig)
    
    imageio.mimsave(output_gif, images, duration=5)


def generate_timestamps2(start_datetime, end_datetime):
    start_time = datetime.strptime(start_datetime, '%Y%m%d%H%M%S')
    end_time = datetime.strptime(end_datetime, '%Y%m%d%H%M%S')
    
    timestamps = []
    current_time = start_time
    while current_time <= end_time:
        timestamps.append(current_time)
        timestamps.append(current_time + timedelta(minutes=30))
        current_time += timedelta(hours=1)
    
    return timestamps

In [24]:
# Example usage
generate_gif_imerg2(
    imerg_dir= "D:/Ghana/IMERG_data/GPM/NRT/IMERG/IMERG-ER", #"D:/Ghana/IMERG_data/GPM/RS/V07/IMERG/IMERG-FR",
    start_datetime="20220722120000",
    end_datetime= "20220723050000",
    dataset_path='Grid/precipitationCal',
    version='C',
    output_gif='imerg_early_Yendi_zoomed.gif',
    plot_tahmo=False,
    plot_gmet=True,
    tahmo_stations=[],
    gmet_stations=['Nakpamboni', 'Yendi'],
    # lat_bounds=(np.min(lat_model), np.max(lat_model)),
    # lon_bounds=(np.min(lon_model), np.max(lon_model))
    lat_bounds=(9.3, 9.8),
    lon_bounds=(-0.5, 0.5)
)

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_25596\758586117.py:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_25596\758586117.py:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_25596\758586117.py:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or

In [34]:
generate_gif_model2(
    model_dir= "D:/Ghana/Output_data/experiments_adapted129/repotest/testOutput", #"D:/Ghana/Output_data/experiments_adapted130/repotest/testOutput", # 
    start_datetime="20220722120000",
    end_datetime="20220723050000",
    variable_name='y_hat',
    output_gif='Model_MSE_Yendi_zoomed.gif',
    plot_tahmo=False,
    plot_gmet=True,
    tahmo_stations=[],
    gmet_stations=['Nakpamboni', 'Yendi'],
    # lat_bounds=(np.min(lat_model), np.max(lat_model)),
    # lon_bounds=(np.min(lon_model), np.max(lon_model))
    lat_bounds=(9.3, 9.8),
    lon_bounds=(-0.5, 0.5)
)

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_25596\758586117.py:43: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_25596\758586117.py:43: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_25596\758586117.py:43: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or

In [10]:
# Example usage
generate_gif_imerg(
    imerg_dir= "D:/Ghana/IMERG_data/GPM/NRT/IMERG/IMERG-ER", #"D:/Ghana/IMERG_data/GPM/RS/V07/IMERG/IMERG-FR", #"D:/Ghana/IMERG_data/GPM/NRT/IMERG/IMERG-ER", #"
    date="20221007",
    time_range=(14, 23),
    dataset_path = 'Grid/precipitationCal',
    version='C',
    output_gif = 'imerg_early_Akim_Oda_zoomed.gif',
    plot_tahmo=True,
    plot_gmet=True, 
    tahmo_stations = ['TA00302', 'TA00045'], 
    gmet_stations = ['Akosombo', 'Akuse', 'Akim_Oda'],
    # lat_bounds=(np.min(lat_model), np.max(lat_model)),
    # lon_bounds=(np.min(lon_model), np.max(lon_model))
    lat_bounds=(5.8, 6.6),
    lon_bounds=(-1.2, 0.4)
)

2022-10-07 14:00:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_22300\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-10-07 14:30:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_22300\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-10-07 15:00:00


...

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_22300\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-10-07 23:00:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_22300\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-10-07 23:30:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_22300\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


In [12]:
# Example usage
generate_gif_model(
    model_dir= "D:/Ghana/Output_data/experiments_adapted129/repotest/testOutput", #"D:/Ghana/Output_data/experiments_adapted130/repotest/testOutput", #
    date="20221007",
    time_range=(14, 23),
    variable_name = 'y_hat',
    output_gif = 'Model_MSE_Akim_Oda_zoom.gif',
    plot_tahmo=True,
    plot_gmet=True, 
    tahmo_stations = ['TA00302', 'TA00045'], 
    gmet_stations = ['Akosombo', 'Akuse', 'Akim_Oda'],
    # lat_bounds=(np.min(lat_model), np.max(lat_model)),
    # lon_bounds=(np.min(lon_model), np.max(lon_model))
    lat_bounds=(5.8, 6.6),
    lon_bounds=(-1.2, 0.4)
)

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_10352\1939745950.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_10352\1939745950.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_10352\1939745950.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio`

In [26]:
# Example usage
generate_gif_imerg(
    imerg_dir= "D:/Ghana/IMERG_data/GPM/RS/V07/IMERG/IMERG-FR", #"D:/Ghana/IMERG_data/GPM/NRT/IMERG/IMERG-ER", #"D:/Ghana/IMERG_data/GPM/NRT/IMERG/IMERG-ER", #"
    date="20220521",
    time_range=(14, 23),
    dataset_path = 'Grid/precipitation',
    version='C',
    output_gif = 'imerg_final_Accra.gif',
    plot_tahmo=True,
    plot_gmet=True, 
    tahmo_stations = ['TA00016', 'TA00391', 'TA00118'], 
    gmet_stations = ['Tema'],
    lat_bounds=(np.min(lat_model), np.max(lat_model)),
    lon_bounds=(np.min(lon_model), np.max(lon_model))
    # lat_bounds=(5.6, 6),
    # lon_bounds=(-0.5, 0.3)
)

2022-05-21 14:00:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-05-21 14:30:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-05-21 15:00:00


...

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-05-21 23:00:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-05-21 23:30:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


In [22]:
# Example usage
generate_gif_model(
    model_dir= "D:/Ghana/Output_data/experiments_adapted129/repotest/testOutput", #"D:/Ghana/Output_data/experiments_adapted130/repotest/testOutput", # 
    date="20220521",
    time_range=(14, 23),
    variable_name = 'y_hat',
    output_gif = 'Model_MSE_Accra.gif',
    plot_tahmo=True,
    plot_gmet=True, 
    tahmo_stations = ['TA00016', 'TA00391', 'TA00118'], 
    gmet_stations = ['Tema'],
    lat_bounds=(np.min(lat_model), np.max(lat_model)),
    lon_bounds=(np.min(lon_model), np.max(lon_model))
    # lat_bounds=(5.6, 6),
    # lon_bounds=(-0.5, 0.3)
)

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_10352\1939745950.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_10352\1939745950.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_10352\1939745950.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio`

In [22]:
# Example usage
generate_gif_model(
    model_dir= "D:/Ghana/Output_data/experiments_adapted130/repotest/testOutput", #"D:/Ghana/Output_data/experiments_adapted129/repotest/testOutput",  
    date="20220424",
    time_range=(14, 23),
    variable_name = 'y_hat',
    output_gif = 'Model_balanced_Axim.gif',
    plot_tahmo=True,
    plot_gmet=True, 
    tahmo_stations = ['TA00120', 'TA00121'], 
    gmet_stations = ['Axim'],
    lat_bounds=(np.min(lat_model), np.max(lat_model)),
    lon_bounds=(np.min(lon_model), np.max(lon_model))
    # lat_bounds=(4.5, 5.2),
    # lon_bounds=(-3, -1)
)

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or

In [32]:
# Example usage
generate_gif_imerg(
    imerg_dir= "D:/Ghana/IMERG_data/GPM/NRT/IMERG/IMERG-ER", #"D:/Ghana/IMERG_data/GPM/RS/V07/IMERG/IMERG-FR", #"D:/Ghana/IMERG_data/GPM/NRT/IMERG/IMERG-ER", #"
    date= "20220424",
    time_range=(14, 23),
    dataset_path = 'Grid/precipitationCal',
    version='B',
    output_gif = 'imerg_early_Axim.gif',
    plot_tahmo=True,
    plot_gmet=True, 
    tahmo_stations = ['TA00120', 'TA00121'], 
    gmet_stations = ['Axim'],
    lat_bounds=(np.min(lat_model), np.max(lat_model)),
    lon_bounds=(np.min(lon_model), np.max(lon_model))
    # lat_bounds=(4.5, 5.2),
    # lon_bounds=(-3, -1)
)

2022-04-24 14:00:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-04-24 14:30:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-04-24 15:00:00


...

C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-04-24 23:00:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))


2022-04-24 23:30:00


C:\Users\c.kwa\AppData\Local\Temp\ipykernel_5068\3370927687.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_image))
